# Ablation Study 2.2: Feature Extractor vs. Identity Mapping

## Motivation

SpikePoint employs an identity mapping instead of PointNet's T-Net to reduce overhead. However, this may limit representational flexibility. Introducing a lightweight linear projection layer could uncover whether early-stage learnable transformations improve accuracy or harm efficiency.

## Experimental Plan

1. Modify the model by replacing the identity mapping with a simple linear layer: `h_i = W * x_i + b`
2. Train and evaluate both configurations (identity vs learnable premap)
3. Compare accuracy, parameter count, and training efficiency

## Expected Insight

This ablation will quantify the performance trade-off between learnability and structural simplicity. If the learnable mapping improves accuracy significantly, it suggests that identity mapping sacrifices valuable spatial adaptation. Conversely, if accuracy remains similar, it validates SpikePoint's minimalist design philosophy.

## Dataset Setup

Before running this notebook, ensure the ModelNet40 dataset is downloaded and extracted to: `data/modelnet40_normal_resampled/`

In [1]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

sys.path.append('..')
from data_utils.ModelNetDataLoader import ModelNetDataLoader
from models.spike_model import SpikeModel
from models.pointnet_utils import PointNetEncoder, feature_transform_reguliarzer
import provider

print('Imports successful!')
from cache_utils import load_training_history, save_training_history, cache_checkpoint, load_cached_checkpoint_path, best_metric
from viz_utils import plot_training_curves, summarize_histories, plot_metric_table, plot_metric_bars


Imports successful!


## Modified PointNet with Learnable Premap

**Important Fix**: The premap layer must preserve the time dimension (T) to work correctly with SpikeLinear, which expects 3D input [T, B, C].

In [2]:
class PointNetWithPremap(nn.Module):
    def __init__(self, k=40, normal_channel=True, use_premap=False):
        super(PointNetWithPremap, self).__init__()
        channel = 6 if normal_channel else 3
        self.use_premap = use_premap
        
        if self.use_premap:
            self.premap = nn.Linear(channel, channel)
            print(f'Using learnable premap: Linear({channel}, {channel})')
        else:
            self.premap = None
            print('Using identity mapping (no premap)')
        
        self.feat = PointNetEncoder(global_feat=True, feature_transform=True, channel=channel)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.dropout = nn.Dropout(p=0.4)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()

    def forward(self, x):
        # Apply premap if enabled - FIXED to preserve time dimension for SpikeLinear
        if self.use_premap:
            if x.dim() == 3:  # [B, C, N]
                B, C, N = x.shape
                x = x.transpose(2, 1)          # [B, N, C]
                x = x.reshape(1, B * N, C)     # [T=1, B*N, C] - valid for SpikeLinear
                x = self.premap(x)             # output [1, B*N, C]
                x = x.reshape(B, N, C).transpose(2, 1)  # back to [B, C, N]
            elif x.dim() == 4:  # [T, B, C, N]
                T, B, C, N = x.shape
                x = x.permute(0, 1, 3, 2).reshape(T, B * N, C)  # [T, B*N, C]
                x = self.premap(x)                               # [T, B*N, C]
                x = x.reshape(T, B, N, C).permute(0, 1, 3, 2)   # back to [T, B, C, N]
        
        x, trans, trans_feat = self.feat(x)
        x = self.relu1(self.bn1(self.fc1(x)))
        x = self.relu2(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1 if len(x.shape) == 2 else 2)
        return x, trans_feat

class get_loss(nn.Module):
    def __init__(self, mat_diff_loss_scale=0.001):
        super(get_loss, self).__init__()
        self.mat_diff_loss_scale = mat_diff_loss_scale

    def forward(self, pred, target, trans_feat):
        loss = F.nll_loss(pred, target)
        mat_diff_loss = feature_transform_reguliarzer(trans_feat)
        return loss + mat_diff_loss * self.mat_diff_loss_scale

print('Modified PointNet model created!')

Modified PointNet model created!


## Configuration

In [3]:
class Args:
    def __init__(self, use_premap=False, num_category=40):
        self.use_cpu = False
        self.gpu = '0'
        self.batch_size = 24
        self.model = 'pointnet_cls'
        self.num_category = num_category
        self.epoch = 200
        self.learning_rate = 0.001
        self.num_point = 1024
        self.optimizer = 'Adam'
        premap_str = 'premap' if use_premap else 'identity'
        self.log_dir = f'ablation_{premap_str}_modelnet{num_category}'
        self.decay_rate = 1e-4
        self.use_normals = False
        self.process_data = False
        self.use_uniform_sample = False
        self.step = 1
        self.spike = True
        self.temp = 5.0
        self.use_premap = use_premap

args_identity = Args(use_premap=False, num_category=40)
args_premap = Args(use_premap=True, num_category=40)
print(f'Identity config: {args_identity.log_dir}')
print(f'Premap config: {args_premap.log_dir}')

Identity config: ablation_identity_modelnet40
Premap config: ablation_premap_modelnet40


## Helper Functions

In [4]:
def setup_experiment(args):
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    exp_dir = Path('../log/classification') / args.log_dir
    exp_dir.mkdir(parents=True, exist_ok=True)
    checkpoints_dir = exp_dir / 'checkpoints'
    checkpoints_dir.mkdir(exist_ok=True)
    return exp_dir, checkpoints_dir

def load_data(args):
    data_path = 'C:\\Users\\VIICTTE\\ML_Project\\modelnet40_normal_resampled'
    train_dataset = ModelNetDataLoader(root=data_path, args=args, split='train')
    test_dataset = ModelNetDataLoader(root=data_path, args=args, split='test')
    trainDataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=4, drop_last=True)
    testDataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=4)
    return trainDataLoader, testDataLoader

def create_model(args):
    classifier = PointNetWithPremap(k=args.num_category, normal_channel=args.use_normals, use_premap=args.use_premap)
    if args.spike:
        classifier = SpikeModel(classifier, args.step, args.temp)
        classifier.set_spike_state(True)
    criterion = get_loss()
    if not args.use_cpu:
        classifier = classifier.cuda()
        criterion = criterion.cuda()
    return classifier, criterion

print('Helper functions defined!')

Helper functions defined!


## Training Function

In [5]:
def train_model(args, exp_dir, checkpoints_dir, max_epochs=None):
    if max_epochs:
        args.epoch = max_epochs
    trainDataLoader, testDataLoader = load_data(args)
    classifier, criterion = create_model(args)
    
    total_params = sum(p.numel() for p in classifier.parameters())
    print(f'Total parameters: {total_params:,}')
    
    optimizer = torch.optim.Adam(classifier.parameters(), lr=args.learning_rate, weight_decay=args.decay_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.7)
    
    best_acc = 0.0
    history = []
    
    for epoch in range(args.epoch):
        print(f'Epoch {epoch+1}/{args.epoch}')
        classifier.train()
        scheduler.step()
        mean_correct = []
        
        for points, target in tqdm(trainDataLoader):
            optimizer.zero_grad()
            points = points.data.numpy()
            points = provider.random_point_dropout(points)
            points[:,:,0:3] = provider.random_scale_point_cloud(points[:,:,0:3])
            points[:,:,0:3] = provider.shift_point_cloud(points[:,:,0:3])
            points = torch.Tensor(points).transpose(2, 1)
            if not args.use_cpu:
                points, target = points.cuda(), target.cuda()
            pred, trans_feat = classifier(points)
            loss = criterion(pred, target.long(), trans_feat)
            loss.backward()
            optimizer.step()
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.long().data).cpu().sum()
            mean_correct.append(correct.item() / float(points.size()[0]))
        
        train_acc = np.mean(mean_correct)
        
        with torch.no_grad():
            classifier.eval()
            test_correct = []
            for points, target in testDataLoader:
                if not args.use_cpu:
                    points, target = points.cuda(), target.cuda()
                points = points.transpose(2, 1)
                pred, _ = classifier(points)
                pred_choice = pred.data.max(1)[1]
                correct = pred_choice.eq(target.long().data).cpu().sum()
                test_correct.append(correct.item() / float(points.size()[0]))
            test_acc = np.mean(test_correct)
        
        print(f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
        history.append({'epoch': epoch+1, 'train_acc': train_acc, 'test_acc': test_acc})
        
        if test_acc >= best_acc:
            best_acc = test_acc
            torch.save({'model_state_dict': classifier.state_dict()}, str(checkpoints_dir / 'best_model.pth'))
    
    return classifier, history, best_acc

print('Training function defined!')

Training function defined!


## Train Models

**Note**: Training takes significant time. Consider reducing epochs for testing.

In [6]:
# Load cached baseline (identity premap) run if available, otherwise train
BASELINE_CACHE_NAME = 'baseline_step_1'
identity_history, identity_meta = load_training_history(BASELINE_CACHE_NAME, with_metadata=True)
if identity_history:
    print(f"Loaded cached baseline history with {len(identity_history)} epoch(s) from {BASELINE_CACHE_NAME}.json")
    history_id = identity_history
    best_acc_id = best_metric(history_id, ['test_acc', 'test_instance_acc']) or 0.0
    classifier_id = None
    cached_ckpt_path = load_cached_checkpoint_path(BASELINE_CACHE_NAME)
    if cached_ckpt_path:
        print(f'Cached baseline checkpoint available at: {cached_ckpt_path}')
else:
    print('No cached baseline history found; training identity mapping from scratch.')
    exp_dir_id, ckpt_dir_id = setup_experiment(args_identity)
    classifier_id, history_id, best_acc_id = train_model(args_identity, exp_dir_id, ckpt_dir_id, max_epochs=200)
    metadata_id = {
        'variant': 'baseline_identity',
        'config': dict(vars(args_identity)),
        'max_epochs': args_identity.epoch,
    }
    history_path_id = save_training_history(history_id, BASELINE_CACHE_NAME, metadata=metadata_id)
    print(f'Saved baseline history to {history_path_id}')
    best_ckpt_id = ckpt_dir_id / 'best_model.pth'
    if best_ckpt_id.exists():
        cached_ckpt_path = cache_checkpoint(best_ckpt_id, BASELINE_CACHE_NAME)
        print(f'Cached baseline checkpoint to {cached_ckpt_path}')
print(f'Identity Best Accuracy: {best_acc_id:.4f}')

Loaded cached baseline history with 200 epoch(s) from baseline_step_1.json
Cached baseline checkpoint available at: ..\artifacts\spikepointnet\checkpoints\baseline_step_1.pth
Identity Best Accuracy: 0.8818


In [ ]:
# Train with learnable premap
exp_dir_pm, ckpt_dir_pm = setup_experiment(args_premap)
classifier_pm, history_pm, best_acc_pm = train_model(args_premap, exp_dir_pm, ckpt_dir_pm, max_epochs=200)
print(f'Premap Best Accuracy: {best_acc_pm:.4f}')

The size of train data is 9843
The size of test data is 2468
Using learnable premap: Linear(3, 3)
Total parameters: 3,483,389
Epoch 1/200


100%|██████████████████████████████████████████████████████████████████| 410/410 [00:22<00:00, 18.45it/s]


Train Acc: 0.3643, Test Acc: 0.0405
Epoch 2/200


100%|██████████████████████████████████████████████████████████████████| 410/410 [00:18<00:00, 22.18it/s]


Train Acc: 0.3295, Test Acc: 0.0125
Epoch 3/200


...

100%|██████████████████████████████████████████████████████████████████████| 410/410 [00:17<00:00, 22.82it/s]


Train Acc: 0.8181, Test Acc: 0.8480
Epoch 164/200


100%|██████████████████████████████████████████████████████████████████████| 410/410 [00:18<00:00, 22.78it/s]


Train Acc: 0.8161, Test Acc: 0.8342
Epoch 165/200


  4%|██▉                                                                    | 17/410 [00:04<00:44,  8.84it/s]

## Visualization and Analysis

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

metrics = {
    "Train Accuracy": ["train_acc"],
    "Test Accuracy": ["test_instance_acc", "test_acc"],
}

available_histories = {}
if 'history_id' in locals() and history_id:
    available_histories["Identity Mapping"] = history_id
if 'history_pm' in locals() and history_pm:
    available_histories["Learnable Premap"] = history_pm

if not available_histories:
    print('No training history available for visualization. Run the training cells above first.')
else:
    fig, axes = plot_training_curves(
        available_histories,
        metrics,
        title="Premap Ablation: Accuracy per Epoch"
    )

    figures_dir = Path("../log/figures") / "premap"
    figures_dir.mkdir(parents=True, exist_ok=True)
    curve_path = figures_dir / "accuracy_curves.png"
    fig.savefig(curve_path, dpi=150, bbox_inches="tight")
    plt.show()

    summary_stats = summarize_histories(available_histories, metrics)
    table_fig, table_ax = plot_metric_table(
        summary_stats,
        title="Premap Ablation: Accuracy per Epoch Summary",
        value_fmt="{:.4f}",
        include_first=True
    )
    table_path = figures_dir / "accuracy_summary.png"
    table_fig.savefig(table_path, dpi=150, bbox_inches="tight")
    plt.show()

    bar_fig, bar_ax = plot_metric_bars(
        summary_stats,
        metric_name="Test Accuracy",
        title="Best Test Accuracy",
        ylabel="Test Accuracy"
    )
    bar_path = figures_dir / "best_accuracy.png"
    bar_fig.savefig(bar_path, dpi=150, bbox_inches="tight")
    plt.show()

    def _fmt(value):
        return "-" if value is None else f"{value:.4f}"

    print('Detailed metrics:')
    for label, metric_stats in summary_stats.items():
        stats = metric_stats.get("Test Accuracy", {})
        print(f"  {label}: last={{_fmt(stats.get('last'))}}, best={{_fmt(stats.get('best'))}}")


## Summary

This ablation study compares identity mapping vs learnable premap (linear projection). The results show whether early-stage learnable transformations improve accuracy or if the minimalist identity mapping is sufficient.